In [ ]:
# update with login token for your cluster
!oc login --token=shaXXX --server=https://XXXX

In [ ]:
%%writefile tfjob-mnist.yaml
apiVersion: kubeflow.org/v1
kind: TFJob
metadata:
  generateName: tfjob-multi-
  namespace: tf-distributed
spec:
  cleanPodPolicy: Running
  tfReplicaSpecs:
    Worker:
      replicas: 2
      restartPolicy: Never
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:            
          containers:
          - name: tensorflow
            image: "quay.io/selbi/ubi-cuda-tf-mnist:1.3"
            command: ["python3", "/home/mnist/src/main.py"] 
            imagePullPolicy: Always
            env:
              - name: TF_FORCE_GPU_ALLOW_GROWTH
                value: "true"
            resources:
                limits:
                    nvidia.com/gpu: 3

In [ ]:
!oc new-project tf-distributed

In [ ]:
!oc create -f tfjob-mnist.yaml

In [ ]:
!oc get pods | grep tfjob-multi

In [ ]:
# update the pod name with output above
!oc logs pod/tfjob-multi-nqtqm-worker-0 | grep "/job:worker/replica:0/task:0/device:GPU:"

In [ ]:
# update the pod name with output from "!oc get pods..."
!oc logs pod/tfjob-multi-nqtqm-worker-0 | grep "GrpcChannelCache"

In [ ]:
# update the pod name with output from "!oc get pods..."
!oc logs pod/tfjob-multi-qhqhh-worker-0 | grep "accuracy"

In [ ]:
# once training is completed, clean up and delete the project and all related resources
!oc delete tfjobs --all -n tf-distributed
!oc delete project tf-distributed

In [ ]:
# ADVANCED TOPIC: How to check available NVIDIA GPUs on the OpenShift cluster and monitor their usage
!oc get pods -n gpu-operator-resources -w

In [ ]:
# from the output of the cell above, replace 'XXXX' of the command below with corresponding pod name
!oc exec -it nvidia-driver-daemonset-XXXX nvidia-smi -n gpu-operator-resources